In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [4]:
#data = pd.read_csv('kddcup.data_10_percent_corrected')
labels = pd.read_csv("labels.csv",sep = ":",header = None)
data   = pd.read_csv("kddcup.data_10_percent_corrected", names = labels.iloc[:,0].values)
print(data.shape)

(494021, 42)


In [5]:
print(data.head())

   duration protocol_type service flag  src_bytes  dst_bytes  land  \
0         0           tcp    http   SF        181       5450     0   
1         0           tcp    http   SF        239        486     0   
2         0           tcp    http   SF        235       1337     0   
3         0           tcp    http   SF        219       1337     0   
4         0           tcp    http   SF        217       2032     0   

   wrong_fragment  urgent  hot   ...     dst_host_srv_count  \
0               0       0    0   ...                      9   
1               0       0    0   ...                     19   
2               0       0    0   ...                     29   
3               0       0    0   ...                     39   
4               0       0    0   ...                     49   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                     1.0                     0.0   
1                     1.0                     0.0   
2                     1.0                

In [ ]:
data.iloc[:,41] = np.where(data.iloc[:,41] != 'normal.',1,data.iloc[:,41])
data.iloc[:,41] = np.where(data.iloc[:,41] == 'normal.',0,data.iloc[:,41])

data.iloc[:,41].unique()

In [ ]:
lb_make = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
cat_columns = labels.loc[labels.iloc[:,1] == " symbolic.",0].values
data[cat_columns] = data[cat_columns].apply(lambda col: lb_make.fit_transform(col))

In [ ]:
def ffr(df, t):
    classes = ['normal', 'u2r', 'dos', 'r2l', 'probe']
    classes2 = [0,1]
    df_new = df.iloc[:,:-1]
    var_d = []
    for clas in classes2:
        temp = df.loc[df['output'] == clas]
        #print("is unique ", temp.output.unique())
        temp = temp.iloc[:,:-1].values
        #print("temp = ", temp)
        min_max_scaler = preprocessing.MinMaxScaler()
        temp_scaled = min_max_scaler.fit_transform(temp)
        #print("temp_scaled = ", temp_scaled)
        mean_ind = temp_scaled.mean(axis = 0)
        #print("mean_ind = ", mean_ind)
        var_d_f = np.square(mean_ind - temp).mean(axis = 0)
        #print("var_d_f_f = ", var_d_f)
        var_d.append(var_d_f)
    var_d = np.array(var_d)
    #print(var_d)
    var_d_mean = var_d.mean(axis = 0)
    
    
#     var = np.zeros(len(mean_means))
#     for i in means:
#         var += np.square(i - mean_means)
#     var /= len(mean_means)
#     #print(var)
    indexes = list(np.argsort(var_d_mean))
    filtered_indexes = indexes[:t]
    filtered_indexes.append(41)
    filtered_data = df.iloc[:, filtered_indexes]
    return filtered_data

In [ ]:
data = ffr(data,30)
print("With FFS with number of features : ",30)

In [ ]:
print(data.shape)
Y = data.iloc[:,-1]
Y = np.array(Y)
#Y = Y.reshape(Y.shape[0],1)
print(Y.shape)
data.drop(data.columns[-1], axis=1, inplace=True)
print(data.shape)

In [ ]:
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(data, Y, test_size = 0.20) 

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
#Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_test = np.array(Y_test)
#Y_test = Y_test.reshape(Y_test.shape[0],1)

print("X_train : ",X_train.shape)
print("X_test : ",X_test.shape)
print("Y_train : ",Y_train.shape)
print("Y_test : ",Y_test.shape)

In [ ]:
# Preprocessing in order to do the computation faster doing scaling can use MinMaxScaler as well...
# Check how SVM learns in terms of distance calculation


from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


X_train = X_train.astype(float)
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
print("X_train Shape : ", X_train.shape)

Y_train = np.array(Y_train)
encoder = LabelEncoder()
encoder.fit(Y_train)
Y_train = encoder.transform(Y_train)
print("Y_train Shape : ",Y_train.shape)

X_test = X_test.astype(float)
scaler = preprocessing.StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)
print("X_train Shape : ", X_train.shape)

Y_test = encoder.transform(Y_test)
print("Y_test Shape : ",Y_test.shape)


In [ ]:
from sklearn import svm
import time

svm_model = svm.SVC(kernel='linear',C=1.0)

start = time.time()
svm_model.fit(X_train,Y_train)
end = time.time()

print("Training Time for Binary SVM : ", (end - start))

# Predicting the Results

predicted = svm_model.predict(X_test)

print("Done Predicting..")


predicted = predicted.ravel()
#print(len(predicted))
Y_predicted = np.empty((len(predicted),))
for i in range(len(predicted)):
    if predicted[i] < 0.5:
        Y_predicted[i] = 0
        #Y_predicted[i] = (Y_predicted[i])
    else:
        Y_predicted[i] = 1
        #Y_predicted[i] = math.floor(Y_predicted[i])

Y_predicted = Y_predicted.astype(int)
Y_test_final = np.array(Y_test)
print(Y_predicted.shape)
print(Y_test_final.shape)
print(type(Y_predicted))
print(type(Y_test_final))

print("Y_predicted Unique and its Frequency :")
y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
print(y_pre,y_pred_counts)

print("Y_test Final unique and its Frequency : ")
y_test_final,y_test_counts = np.unique(Y_test_final,return_counts=True)
print(y_test_final,y_test_counts)


tp,fp,tn,fn = 0,0,0,0
for i in range(len(Y_test_final)):
    if Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 1:
        tp = tp + 1
    elif Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 0:
        tn = tn + 1
    elif Y_test_final[i] != Y_predicted[i] and Y_test_final[i] == 0:
        fp = fp + 1
    else:
        fn = fn + 1

accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall)/(precision + recall)
print("tp :",tp)
print("tn :",tn)
print("fp :",fp)
print("fn :",fn)
print("Accuracy : ",accuracy)
print("Precision : ",precision)
print("Recall : ",recall)
print("f1_score : ",f1_score)



In [ ]:
## RBF Kernel

svm_model = svm.SVC(kernel='rbf',C=1.0)
svm_model.fit(X_train,Y_train)
print("Done Modelling...")

# Predicting the Results

predicted = svm_model.predict(X_test)

print("Done Predicting..")


predicted = predicted.ravel()
#print(len(predicted))
Y_predicted = np.empty((len(predicted),))
for i in range(len(predicted)):
    if predicted[i] < 0.5:
        Y_predicted[i] = 0
        #Y_predicted[i] = (Y_predicted[i])
    else:
        Y_predicted[i] = 1
        #Y_predicted[i] = math.floor(Y_predicted[i])

Y_predicted = Y_predicted.astype(int)
Y_test_final = np.array(Y_test)
print(Y_predicted.shape)
print(Y_test_final.shape)
print(type(Y_predicted))
print(type(Y_test_final))

print("Y_predicted Unique and its Frequency :")
y_pre,y_pred_counts = np.unique(Y_predicted,return_counts=True)
print(y_pre,y_pred_counts)

print("Y_test Final unique and its Frequency : ")
y_test_final,y_test_counts = np.unique(Y_test_final,return_counts=True)
print(y_test_final,y_test_counts)


tp,fp,tn,fn = 0,0,0,0
for i in range(len(Y_test_final)):
    if Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 1:
        tp = tp + 1
    elif Y_test_final[i] == Y_predicted[i] and Y_test_final[i] == 0:
        tn = tn + 1
    elif Y_test_final[i] != Y_predicted[i] and Y_test_final[i] == 0:
        fp = fp + 1
    else:
        fn = fn + 1

accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = (2*precision*recall)/(precision + recall)
print("tp :",tp)
print("tn :",tn)
print("fp :",fp)
print("fn :",fn)
print("Accuracy : ",accuracy)
print("Precision : ",precision)
print("Recall : ",recall)
print("f1_score : ",f1_score)
